# **Model Server**

_______________________________________________________________________________

<a id="top"></a>
## **steps**
**[notebook installs](#installs)**<br>
**[nuclio code section](#nuclio)**<br>
    - [inference server](#server)<br>
**[deploy](#deploy)**<br>
**[test deployment](#test)**<br>
**[test saved model object](#testingoutside)**


<a id="installs"></a>
_______________________________________________________________________________

## **notebook installs**

These are the packages we'll need to run this notebook, please install them once:

    !pip install -U -q kfserving==0.2.0 tensorflow==2.0.0b1 keras pandas 
    !pip install -U -q kubernetes==9.0.0
    !pip install -U -q azure

    !pip uninstall -y mlrun
    !pip install -q git+https://github.com/mlrun/mlrun.git@development
    !pip install -U -q git+https://github.com/yjb-ds/functions-demo.git

<a id="nuclio"></a>
_______________________________________________________________________________

## **nuclio code section**

If you have never run nuclio functions in your notebooks, please uncomment and run the following:

    !pip install nuclio-jupyter

In [1]:
# nuclio: ignore
import nuclio

Install the following packages so they are available to the function:

In [2]:
%%nuclio cmd -c
pip install kfserving==0.2.0
pip install numpy==1.16.4 
pip install tensorflow==2.0.0b1
pip install pandas==0.25.3
pip install azure
pip install joblib
pip install git+https://github.com/mlrun/mlrun.git@development
pip install git+https://github.com/yjb-ds/functions-demo.git

In [3]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

import kfserving
import numpy as np
from pickle import load
from typing import List
from tensorflow.keras.models import Sequential

In [4]:
TARGET_PATH = '/User/mlrun/simdata'
MODEL_NAME = 'model.pkl'

In [5]:
class MyKerasClassifier(kfserving.KFModel):
    def __init__(self,
                 name: str,
                 model: str,
                 classifier = None):
        """TFKerasClassifier
        
        Server model wrapper.
        
        :param name:            model name
        :param model:       path of stored model
        :param classifier:      class type of classifier model
        
        """
        super().__init__(name)
        self.name = name
        self.model = model
        if classifier:
            self.ready = True

    def load(self):
        """Load model from storage.
        
        Note that loading to load a model we specify only a folder. Our custom
        loader takes care of the details.
        """
        self.classifier = load(self.model)
    
    def predict(self, body):
        """Generate model predictions from sample.
        
        :param body: A list of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        try:
            return body
#             feats = np.asarray(body['instances'])
#             result: np.ndarray = self.classifier.predict(feats)
#             return result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")

In [6]:
# nuclio: end-code

<a id="deploy"></a>
_______________________________________________________________________________

## **deploy**

In [7]:
from mlrun import new_model_server, mount_v3io

In [8]:
TARGET_PATH = "/User/mlrun/simdata"
MODEL_NAME = "model.pkl"

In [9]:
fn = new_model_server(MODEL_NAME, 
                      models={"tfkeras_joblib": TARGET_PATH}, 
                      model_class="MyKerasClassifier").apply(mount_v3io())

While debugging your project code, you may want to set the following flag to `True` in order to ensure that all the layers in your image get rebuilt and your changes included:

In [10]:
fn.spec.no_cache=True

After running the following cell, you can start watching the function's deployment by selecting **Projects** in the Iguazio platform UI, and clicking on the project name, and selecting the correct function and version:

In [11]:
addr = fn.deploy(project=("github-demos"))

[mlrun] 2020-01-11 17:38:01,790 deploy started
[nuclio] 2020-01-11 17:43:26,938 (error) Failed to deploy. Details:

Error - NuclioFunction in error state (
Error - Service "model.pkl" is invalid: metadata.name: Invalid value: "model.pkl": a DNS-1035 label must consist of lower case alphanumeric characters or '-', start with an alphabetic character, and end with an alphanumeric character (e.g. 'my-name',  or 'abc-123', regex used for validation is '[a-z]([-a-z0-9]*[a-z0-9])?')
    .../nuclio/pkg/platform/kube/functionres/lazy.go:399

Call stack:
Failed to create resource
    .../nuclio/pkg/platform/kube/functionres/lazy.go:399
Failed to create/update service
    .../nuclio/pkg/platform/kube/functionres/lazy.go:202
Failed to create/update function
    .../platform/kube/controller/nucliofunction.go:113
)
    .../nuclio/nuclio/pkg/platform/kube/deployer.go:197

Call stack:
NuclioFunction in error state (
Error - Service "model.pkl" is invalid: metadata.name: Invalid value: "model.pkl": a D

DeployError: cannot deploy 

<a id="deploy"></a>
_______________________________________________________________________________

## **test**

In [ ]:
import pandas as pd
import requests
import json

#### grab 2 rows of data - balanced dataset

In [ ]:
features = pd.read_csv("x_test_50.csv").iloc[:2,:]
labels = pd.read_csv("y_test_50.csv").iloc[:2,:]

In [ ]:
features

In [ ]:
labels

#### create an event and wrap it in json

In [ ]:
event = {"instances" : features.values.tolist()}

If the notebook is restarted for some reason, however the function has already been deployed, simply uncomment the following cell and paste in the original endpoint here.  You can 
retrieve the function's enpoint address from the platform ui under **Projects**. Look for
the project name you gave to the deployment of interest, in our case **mlrun-demos**.
Click on the function in the project, copy the 'Invocation URL' and paste here:

In [ ]:
# addr = "3.137.70.243:31811"

In [ ]:
resp = requests.post(addr + "/tfkeras_joblib/predict", json=event)

In [ ]:
json.loads(resp.content)

### Test of Estimated Model Object 

Here we simply grab the estimated model file created on the **[kubeflow pipeline](kubeflow%20pipeline.ipynb)**, load it, and run a test matrix through it.

In [ ]:
import joblib
import os
model_file = os.path.join(TARGET_PATH, 'model.pkl')
keras_model = joblib.load(open(model_file, 'rb'))

In [ ]:
testvals = np.asarray(features.values)

In [ ]:
keras_model.predict(testvals)